In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc
import time
gc.enable()
# import seaborn as sns
# import matplotlib.pyplot as plt
import warnings
import re
# import geopy
from tqdm import tqdm_notebook as tqdm
import json
import os

warnings.filterwarnings('ignore')

In [2]:
%%time
# Load from preprocessed data
df_2018 = pd.read_feather('../data/Final_Divvy_data_2018.feather')

CPU times: user 102 ms, sys: 356 ms, total: 458 ms
Wall time: 855 ms


In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import widgets
import plotly.graph_objs as go

init_notebook_mode(connected=True)
mapbox_access_token = 'pk.eyJ1IjoibWVydnluMTUyIiwiYSI6ImNqeHpkNWZmdjAxczUzY29hbHVoandyMnUifQ.D-botzm1Hr6Gjs8jqwD5VA'

In [4]:
def _get_latlon_arr(df, conf='low'):
    mm = 7
    dd = 1

    if conf == 'ex_low':
        sub_df = df[(df['month'] == mm) & (df['day'] == dd) 
                    & (df.total_in < df.dp_max*0.5)]
        
    elif conf == 'low':
        sub_df = df[(df['month'] == mm) & (df['day'] == dd) 
                    & (df.total_in < df.dp_max) & (df.total_in >= df.dp_max*0.5)]
        
    elif conf == 'medium':
        sub_df = df[(df['month'] == mm) & (df['day'] == dd) 
                    & (df.total_in < df.dp_max*1.5) & (df.total_in >= df.dp_max)]
        
    elif conf == 'high':
        sub_df = df[(df['month'] == mm) & (df['day'] == dd) 
                    & (df.total_in < df.dp_max*3) & (df.total_in >= df.dp_max*1.5)]
        
    else:
        sub_df = df[(df['month'] == mm) & (df['day'] == dd) 
                    & (df.total_in >= df.dp_max*3)]
    return np.array(sub_df.lat_ave), np.array(sub_df.lon_ave)

In [5]:
save_tmp_data = False
if save_tmp_data:
    sub_df = df_2018[(df_2018['month'] == 7) & (df_2018['day'] == 1)].copy()
    sub_df.to_pickle('../data/temp.pk')
    tmp = pd.read_pickle('../data/temp.pk')

In [6]:
## Function to get map data
def _get_map_data(lat_arr, lon_arr, nm, color='orange', subplot='mapbox', 
                  op=0.5, sz=15, vis=False, sl=True):
    return go.Scattermapbox(
        lat=lat_arr,
        lon=lon_arr,
        text=["<b>Lattitude</b>: {:.3f} <br><b>Longitude</b>: {:.3f} "\
             .format(la, lo) for la, lo in zip(lat_arr, lon_arr)],
        mode='markers',
        hoverinfo = 'text',
        marker=go.scattermapbox.Marker(
            size=sz,
            color=color,
            opacity=op
        ),   
        subplot=subplot,
        visible=vis,
        name=nm,
        showlegend=sl,
    )

In [10]:
# Prepare station demand data
confs = ['ex_low', 'low', 'medium', 'high', 'ex_high']
clrs = ['grey','yellow', 'orange', 'red', 'blue']
name = ['Extremely Low', 'Low', 'Medium', 'High', 'Extremely High']
name_ = ['<b> Extremely Low </b> <br><i> <0.5 &#215; Capacity</i>', 
         '<b> Low </b><br> <i>0.5~1 &#215; Capacity</i>', 
         '<b> Medium </b><br> <i>1~1.5 &#215; Capacity</i>', 
         '<b> High </b><br> <i>1.5~3 &#215; Capacity</i>', 
         '<b> Extremely High </b><br> <i>>3 &#215; Capacity</i>']
clrs = ['#ffffd4', '#fed98e', '#fe9929', '#d95f0e', '#993404']
clrs.reverse()
data = []
data_count = {}
for c, clr, nm, nk in zip(confs, clrs, name, name_):
    latarr, lonarr = _get_latlon_arr(df_2018, c)
    dt = _get_map_data(latarr, lonarr, nm, color=clr, subplot='mapbox', 
                       op=1, sz=15, vis=True, sl=True)
    data_count[nk] = len(latarr)
    data.append(dt)

In [11]:
# Prepare bar chart
cf = []
ct = []

for k, v in data_count.items():
    cf.append(k)
    ct.append(v)
chart = go.Bar(
    x=cf,
    y=ct,
    name='Count',
    hoverinfo='none',
    text=ct,
    textposition='auto',
    outsidetextfont=dict(
        color='white',
        size=15,
    ),
    insidetextfont=dict(
            size=15,
        ),
    marker=dict(
        color=clrs),
    showlegend=False,
)
data.append(chart)

In [12]:
## Style of mixed plot
layout = {
    "xaxis": {
        "anchor": "y",
        "domain": [0.6, 0.95],
        'tickangle': -55,
        'tickfont': dict(
            family='Old Standard TT, serif',
            size=14,
            color='white'
        ),
      },
    "yaxis": {
        "anchor": "x",
        "domain": [0.3, 0.75],
        "showgrid": False,
        'title': "",
        'titlefont': dict(
            family='Arial, sans-serif',
            size=20,
            color='white'
        ),
        'ticks':'',
        'showticklabels': False,
        'tickangle': 0,
        'tickfont': dict(
            family='Old Standard TT, serif',
            size=14,
            color='white'
        ),
      },
    "plot_bgcolor": 'black',
    "paper_bgcolor": 'black',
    'mapbox': go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        domain={'x': [0., 0.5], 'y': [0.1, 0.9]},
        center=go.layout.mapbox.Center(
            lat=41.885, 
            lon=-87.628
        ),
        pitch=50,
        zoom=13,
        style = 'dark'
    ),  
    'showlegend': True,
#     'autosize': True,
    'legend': dict(
        font=dict(
            family='sans-serif',
            size=12,
            color='white'
        ),
#         bgcolor='#E2E2E2',
#         bordercolor='#FFFFFF',
#         borderwidth=2,
        orientation="v", 
        x=0.75, 
        y=0.9),
    'margin': go.layout.Margin(l=20, r=10, b=10, t=10, pad=6),
    
}

In [13]:
figure = {}
figure['data'] = data
figure['layout'] = layout


SHOW = True

if SHOW:
    iplot(figure, config={'displayModeBar': False})
else:
    plot(figure, config={'displayModeBar': False}, filename="final_demo.html")